<a href="https://colab.research.google.com/github/ChrisFJardine/Masterclass/blob/main/CJardineAssignment3__Webcrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import re
import time
import os
from datetime import datetime, timedelta

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def get_date(soup):
  mydate = soup.findAll("div", {"class": "form-group has-feedback date"})
  for x in mydate:
    children = x.findChildren("input" , recursive=False)
    for child in children:
      date = child['value']
  return(date)

In [6]:
def get_previous_date(soup):
  link = soup.findAll("a", {"class": "prev"}, href = True)
  for x in link:
    prev_date = (x['href'].replace('?date=', ''))
  return(prev_date)

In [15]:
def get_news(soup):
  mydivs = soup.findAll("div", {"class": "comp-rich-text teaser-text clearfix"})
  ignore = "ABC News"
  news_item = []
  news = []
  date = get_date(soup)
  for div in mydivs:
    children = div.findChildren("p" , recursive=False)
    for child in children:
      if ignore not in child.get_text():
        news_item = [date, child.get_text()]
        #news.append(child.get_text())
        news.append(news_item)
      else:
        continue
  return(news)

In [ ]:
urlbase = "https://www.abc.net.au/news/archive/?date="
date = '2021-11-27'
news = []
for i in range(4000):
  url = urlbase + date
  try:
    page = requests.get(url)
    print(str(i) + ' : ' + str(page.elapsed.total_seconds()))
  except:
    print('error getting request ' + str(page.status_code))
    date = datetime.strptime(date, '%Y-%m-%d') - timedelta(1) #failed, move on to next day
  if len(page.content) > 1000:
    soup = BeautifulSoup(page.text, 'lxml')
    news.append(get_news(soup))
    date = get_previous_date(soup)
    time.sleep(0.5)

news = pd.DataFrame(news)
news.to_csv('/content/drive/MyDrive/Colab Notebooks/Assignment 3/news.csv', index=True)